# Realizando predições com as melhores configurações para a LSTM

## 1. Importando as bibliotecas necessárias

### Bibliotecas Externas

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')

import matplotlib.pyplot as plt
import seaborn as sns 
sns.set_style("ticks")
plt.rcParams['savefig.dpi'] = 200
plt.rcParams["figure.dpi"] = 125
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

### Bibliotecas do projeto

In [2]:
# arquivos de configurações
from pibic2020.parameters.lstm import *
from pibic2020.models import lstm_model

# para separar em conjuntos de treino, teste e validacao
from pibic2020.tools import timeseries

## 2. Pegando os dados gerados

### Dados brutos

In [3]:
dados_henon = pd.read_csv('../../data/raw/henon.csv')
dados_logistic = pd.read_csv('../../data/raw/logistic.csv')
dados_lorenz = pd.read_csv('../../data/raw/lorenz.csv')
dados_mackeyglass = pd.read_csv('../../data/raw/mackeyglass.csv')

### Separando os vetores que serão utilizados

#### Mapa de Hénon

In [4]:
n_henon = pd.DataFrame(dados_henon, columns=['n']).to_numpy()
n_henon = n_henon.ravel()
x_henon = pd.DataFrame(dados_henon, columns=['x']).to_numpy()
x_henon = x_henon.ravel()

#### Mapa de logístico

In [5]:
n_logistic = pd.DataFrame(dados_logistic, columns=['n']).to_numpy()
n_logistic = n_logistic.ravel()
x_logistic = pd.DataFrame(dados_logistic, columns=['x']).to_numpy()
x_logistic = x_logistic.ravel()

#### Sistema de Lorenz

In [6]:
t_lorenz = pd.DataFrame(dados_lorenz, columns=['t']).to_numpy()
t_lorenz = t_lorenz.ravel()
x_lorenz = pd.DataFrame(dados_lorenz, columns=['x']).to_numpy()
x_lorenz = x_lorenz.ravel()

#### Equações de Mackey-Glass

In [7]:
t_mackeyglass = pd.DataFrame(dados_mackeyglass, columns=['t']).to_numpy()
t_mackeyglass = t_mackeyglass.ravel()
p_mackeyglass = pd.DataFrame(dados_mackeyglass, columns=['p']).to_numpy()
p_mackeyglass = p_mackeyglass.ravel()

## 3. Separando as configurações para cada modelo

### Mapa de Hénon

In [8]:
config_henon = lstm_henon.lstm_henon
k_henon = lstm_best_k.K_best_henon_lstm
k_henon = k_henon[0]

### Mapa logístico

In [9]:
config_logistic = lstm_logistic.lstm_logistic
k_logistic = lstm_best_k.K_best_logistic_lstm
k_logistic = k_logistic[0]

### Sistema de Lorenz

In [10]:
config_lorenz = lstm_lorenz.lstm_lorenz
k_lorenz = lstm_best_k.K_best_lorenz_lstm
k_lorenz = k_lorenz[0]

### Equações de Mackey-Glass

In [11]:
config_mackeyglass = lstm_mackeyglass.lstm_mackeyglass
k_mackeyglass = lstm_best_k.K_best_mackeyglass_lstm
k_mackeyglass = k_mackeyglass[0]

## 4. Criando conjuntos de treino, teste e validação

In [12]:
L = 3
tam_teste = 0.15
tam_val = 0.1

#### Mapa de Hénon

In [13]:
serie_henon = timeseries.SerieTemporal(x_henon, K=k_henon, L=L)
X_henon_treino, X_henon_teste, X_henon_val, y_henon_treino, y_henon_teste, y_henon_val = serie_henon.dividir_treino_teste_validacao(tam_teste=tam_teste,
                                                                                                                                    tam_val=tam_val)

#### Mapa logístico

In [14]:
serie_logistic = timeseries.SerieTemporal(x_logistic, K=k_logistic, L=L)
X_logistic_treino, X_logistic_teste, X_logistic_val, y_logistic_treino, y_logistic_teste, y_logistic_val = serie_logistic.dividir_treino_teste_validacao(tam_teste=tam_teste,
                                                                                                                                    tam_val=tam_val)

#### Sistema de Lorenz

In [15]:
scaler = MinMaxScaler(feature_range=(-1,1))
x_lorenz = scaler.fit_transform(x_lorenz.reshape(-1,1)).reshape(len(x_lorenz), )
serie_lorenz = timeseries.SerieTemporal(x_lorenz, K=k_lorenz, L=L)
X_lorenz_treino, X_lorenz_teste, X_lorenz_val, y_lorenz_treino, y_lorenz_teste, y_lorenz_val = serie_lorenz.dividir_treino_teste_validacao(tam_teste=tam_teste,
                                                                                                                                    tam_val=tam_val)

#### Equações de Mackey-Glass

In [16]:
serie_mackeyglass = timeseries.SerieTemporal(p_mackeyglass, K=k_mackeyglass, L=L)
X_mackeyglass_treino, X_mackeyglass_teste, X_mackeyglass_val, y_mackeyglass_treino, y_mackeyglass_teste, y_mackeyglass_val = serie_mackeyglass.dividir_treino_teste_validacao(tam_teste=tam_teste,
                                                                                                                                    tam_val=tam_val)

## 5. Treinando a rede neural

### Mapa de Hénon

In [17]:
model_henon = lstm_model.ModeloLSTM(input_shape=(k_henon, 1),
                                  name=config_henon['name'])

model_henon.criar_modelo(n_units=config_henon["n_units"],
                         init_mode=config_henon["init_mode"])

model_henon.montar(learning_rate=config_henon["learning_rate"])

In [18]:
model_henon.treinar(X_treino=X_henon_treino,
                    X_val=X_henon_val,
                    y_treino=y_henon_treino,
                    y_val=y_henon_val,
                    batch_size=config_henon["batch_size"],
                    verbose=0)

In [19]:
y_henon_pred = model_henon.predicao(X_henon_teste)

In [20]:
mse = mean_squared_error(y_henon_teste, y_henon_pred)
print("MSE = " + str(mse))

MSE = 0.00036270162885772793


### Mapa logístico

In [21]:
model_logistic = lstm_model.ModeloLSTM(input_shape=(k_logistic, 1),
                                  name=config_logistic['name'])

model_logistic.criar_modelo(n_units=config_logistic["n_units"],
                         init_mode=config_logistic["init_mode"])

model_logistic.montar(learning_rate=config_logistic["learning_rate"])

In [22]:
model_logistic.treinar(X_treino=X_logistic_treino,
                    X_val=X_logistic_val,
                    y_treino=y_logistic_treino,
                    y_val=y_logistic_val,
                    batch_size=config_logistic["batch_size"],
                    verbose=0)

In [23]:
y_logistic_pred = model_logistic.predicao(X_logistic_teste)

In [24]:
mse = mean_squared_error(y_logistic_teste, y_logistic_pred)
print("MSE = " + str(mse))

MSE = 0.0004087685343895958


### Sistema de Lorenz

In [25]:
model_lorenz = lstm_model.ModeloLSTM(input_shape=(k_lorenz, 1),
                                  name=config_lorenz['name'])

model_lorenz.criar_modelo(n_units=config_lorenz["n_units"],
                         init_mode=config_lorenz["init_mode"])

model_lorenz.montar(learning_rate=config_lorenz["learning_rate"])

In [26]:
model_lorenz.treinar(X_treino=X_lorenz_treino,
                    X_val=X_lorenz_val,
                    y_treino=y_lorenz_treino,
                    y_val=y_lorenz_val,
                    batch_size=config_lorenz["batch_size"],
                    verbose=0)

In [27]:
y_lorenz_pred = model_lorenz.predicao(X_lorenz_teste, scaler=scaler)

In [28]:
mse = mean_squared_error(scaler.inverse_transform(y_lorenz_teste), y_lorenz_pred)
print("MSE = " + str(mse))

MSE = 0.009727875237371808


### Equações de Mackey-Glass

In [29]:
model_mackeyglass = lstm_model.ModeloLSTM(input_shape=(k_mackeyglass, 1),
                                  name=config_mackeyglass['name'])

model_mackeyglass.criar_modelo(n_units=config_mackeyglass["n_units"],
                         init_mode=config_mackeyglass["init_mode"])

model_mackeyglass.montar(learning_rate=config_mackeyglass["learning_rate"])

In [30]:
model_mackeyglass.treinar(X_treino=X_mackeyglass_treino,
                    X_val=X_mackeyglass_val,
                    y_treino=y_mackeyglass_treino,
                    y_val=y_mackeyglass_val,
                    batch_size=config_mackeyglass["batch_size"],
                    verbose=0)

In [31]:
y_mackeyglass_pred = model_mackeyglass.predicao(X_mackeyglass_teste)

In [32]:
mse = mean_squared_error(y_mackeyglass_teste, y_mackeyglass_pred)
print("MSE = " + str(mse))

MSE = 4.1841524324483206e-05


## 6. Salvando os dados

### Mapa de Hénon

In [33]:
n_henon_teste = n_henon[-len(y_henon_pred):]
dados_henon = np.column_stack((y_henon_teste, y_henon_pred, n_henon_teste))
dados_henon = pd.DataFrame(dados_henon, columns=['y_teste', 'y_pred', 'n'])
dados_henon.to_csv('../../data/predictions/lstm/henon.csv', index = False, header=True)

### Mapa logístico

In [34]:
n_logistic_teste = n_logistic[-len(y_logistic_pred):]
dados_logistic = np.column_stack((y_logistic_teste, y_logistic_pred, n_logistic_teste))
dados_logistic = pd.DataFrame(dados_logistic, columns=['y_teste', 'y_pred', 'n'])
dados_logistic.to_csv('../../data/predictions/lstm/logistic.csv', index = False, header=True)

### Sistema de Lorenz

In [35]:
n_lorenz_teste = t_lorenz[-len(y_lorenz_pred):]
dados_lorenz = np.column_stack((scaler.inverse_transform(y_lorenz_teste), y_lorenz_pred, n_lorenz_teste))
dados_lorenz = pd.DataFrame(dados_lorenz, columns=['y_teste', 'y_pred', 't'])
dados_lorenz.to_csv('../../data/predictions/lstm/lorenz.csv', index = False, header=True)

### Equações de Mackey-Glass

In [36]:
t_mackeyglass_teste = t_mackeyglass[-len(y_mackeyglass_pred):]
dados_mackeyglass = np.column_stack((y_mackeyglass_teste, y_mackeyglass_pred, t_mackeyglass_teste))
dados_mackeyglass = pd.DataFrame(dados_mackeyglass, columns=['y_teste', 'y_pred', 't'])
dados_mackeyglass.to_csv('../../data/predictions/lstm/mackeyglass.csv', index = False, header=True)

## 7. Salvando os modelos

In [37]:
model_henon.salvar("../../models/lstm/lstm-henon.h5")
model_logistic.salvar("../../models/lstm/lstm-logistic.h5")
model_lorenz.salvar("../../models/lstm/lstm-lorenz.h5")
model_mackeyglass.salvar("../../models/lstm/lstm-mackeyglass.h5")

O modelo foi salvo!
O modelo foi salvo!
O modelo foi salvo!
O modelo foi salvo!
